In [1]:
import os
from pathlib import Path

config_path = Path.home() / ".fiftyone" / "config.global_mongodb.json"
os.environ["FIFTYONE_CONFIG_PATH"] = str(config_path)

In [3]:
import evaluate

from seametrics.fo_utils.utils import fo_to_payload

payload = fo_to_payload(
    dataset="SAILING_DATASET_QA",
    gt_field="ground_truth_det",
    models=["yolov5n6_RGB_D2304-v1_9C"],
    tracking_mode=False,
    sequence_list=["Trip_14_Seq_1", "Trip_14_Seq_2"],
    data_type="thermal",
)

module = evaluate.load("SEA-AI/det-metrics", payload=payload)
module.compute()

/var/folders/62/hw7gk5hj4x7bb1mp9_8rj23w0000gn/T/ipykernel_54524/1322497394.py:4: DeprecationWarning: Call to deprecated function (or staticmethod) fo_to_payload. (
Use the `PayloadProcessor(...).payload` instead.
`fo_to_payload` will be removed in the future.
https://github.com/SEA-AI/seametrics/tree/develop)
  payload = fo_to_payload(
Processing sequences: 100%|██████████| 2/2 [00:00<00:00,  3.53it/s]



##### yolov5n6_RGB_D2304-v1_9C #####
                                             METRIC     tp,     fp,     fn,    dup,    pr,    re,    f1,   supp,    fpi,  nImgs
@[ IoU=0.00      | area=      all | maxDets=100 ] =      0,      0,    189,      0,  0.00,  0.00,  0.00,    189,      0,    146


{'yolov5n6_RGB_D2304-v1_9C': {'params': {'imgIds': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145], 'catIds': [-1], 'iouThrs': array([1.e-10]), 'recThrs': array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
         0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
         0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
         0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39

In [4]:
from seametrics.payload.processor import PayloadProcessor

processor = PayloadProcessor(
    dataset_name="SAILING_DATASET_QA",
    gt_field="ground_truth_det",
    models=["yolov5n6_RGB_D2304-v1_9C"],
    tracking_mode=False,
    sequence_list=["Trip_14_Seq_1", "Trip_14_Seq_2"],
    data_type="thermal",
)

module = evaluate.load("SEA-AI/det-metrics", payload=payload)
module.compute()

Processing sequences: 100%|██████████| 2/2 [00:00<00:00,  2.75it/s]



##### yolov5n6_RGB_D2304-v1_9C #####
                                             METRIC     tp,     fp,     fn,    dup,    pr,    re,    f1,   supp,    fpi,  nImgs
@[ IoU=0.00      | area=      all | maxDets=100 ] =      0,      0,    189,      0,  0.00,  0.00,  0.00,    189,      0,    146


{'yolov5n6_RGB_D2304-v1_9C': {'params': {'imgIds': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145], 'catIds': [-1], 'iouThrs': array([1.e-10]), 'recThrs': array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
         0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
         0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
         0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39

In [5]:
import numpy as np

from seametrics.detection import PrecisionRecallF1Support

predictions = [
    {
        "boxes": np.array(
            [
                [449.3, 197.75390625, 6.25, 7.03125],
                [334.3, 181.58203125, 11.5625, 6.85546875],
            ]
        ),
        "labels": np.array([0, 0]),
        "scores": np.array([0.153076171875, 0.72314453125]),
    }
]

ground_truth = [
    {
        "boxes": np.array(
            [
                [449.3, 197.75390625, 6.25, 7.03125],
                [334.3, 181.58203125, 11.5625, 6.85546875],
            ]
        ),
        "labels": np.array([0, 0]),
        "area": np.array([132.2, 83.8]),
    }
]

metric = PrecisionRecallF1Support()  # default settings
metric.update(preds=predictions, target=ground_truth)
metric.compute()["metrics"]

                                             METRIC     tp,     fp,     fn,    dup,    pr,    re,    f1,   supp,    fpi,  nImgs
@[ IoU=0.50      | area=      all | maxDets=100 ] =      0,      2,      2,      0,  0.00,  0.00,  0.00,      2,      0,      1


{'all': {'range': [0, 10000000000.0],
  'iouThr': '0.50',
  'maxDets': 100,
  'tp': 0,
  'fp': 2,
  'fn': 2,
  'duplicates': 0,
  'precision': 0.0,
  'recall': 0.0,
  'f1': 0,
  'support': 2,
  'fpi': 0,
  'nImgs': 1}}

In [5]:
payload = fo_to_payload(
    dataset="SENTRY_VIDEOS_DATASET_QA",
    gt_field="ground_truth_det",
    models=["volcanic-sweep-3_02_2023_N_LN1_ep288_TRACKER"],
    sequence_list=["Sentry_2022_11_PROACT_CELADON_7.5M_MOB_2022_11_25_12_12_39"],
    tracking_mode=True,
    debug=True,
)

/var/folders/4y/pnmysh0j3mx29k4xfh88xkgw0000gn/T/ipykernel_70225/2038669266.py:1: DeprecationWarning: Call to deprecated function (or staticmethod) fo_to_payload. (
Use the `PayloadProcessor(...).payload` instead.
`fo_to_payload` will be removed in the future.
https://github.com/SEA-AI/seametrics/tree/develop)
  payload = fo_to_payload(
Processing sequences: 100%|██████████| 1/1 [00:01<00:00,  1.78s/it]


In [6]:
# processor.tracking_mode = True
# payload = processor.recompute_payload()

module = evaluate.load("SEA-AI/mot-metrics")
json_metrics = module._compute(payload.to_dict(), max_iou=0.99, debug=True)

gt_field_name:  ground_truth_det
models:  ['volcanic-sweep-3_02_2023_N_LN1_ep288_TRACKER']
sequence_list:  ['Sentry_2022_11_PROACT_CELADON_7.5M_MOB_2022_11_25_12_12_39']
sequence/model:  Sentry_2022_11_PROACT_CELADON_7.5M_MOB_2022_11_25_12_12_39 volcanic-sweep-3_02_2023_N_LN1_ep288_TRACKER
formated_predictions:  [[73, 1, 0.4754209961854182, 0.5085374327732526, 0.017908007629163336, 0.030947896212738635, 1], [74, 1, 0.4754209961854182, 0.5085374327732526, 0.017908007629163336, 0.030947896212738635, 1], [75, 1, 0.4754209961854182, 0.5085374327732526, 0.017908007629163336, 0.030947896212738635, 1], [76, 1, 0.4754209961854182, 0.5085374327732526, 0.017908007629163336, 0.030947896212738635, 1], [77, 1, 0.45304379111179693, 0.502922469409144, 0.015787285110300475, 0.03096241434095859, 1], [78, 1, 0.45304379111179693, 0.502922469409144, 0.015787285110300475, 0.03096241434095859, 1], [79, 1, 0.45304379111179693, 0.502922469409144, 0.015787285110300475, 0.03096241434095859, 1], [80, 1, 0.451940

In [7]:
json_metrics

{'Sentry_2022_11_PROACT_CELADON_7.5M_MOB_2022_11_25_12_12_39': {'volcanic-sweep-3_02_2023_N_LN1_ep288_TRACKER': {'idf1': 0.9441397521836279,
   'idp': 0.9699499165275459,
   'idr': 0.9196675900277008,
   'recall': 0.9333201424614167,
   'precision': 0.9843489148580968,
   'num_unique_objects': 2,
   'mostly_tracked': 1,
   'partially_tracked': 0,
   'mostly_lost': 1,
   'num_false_positives': 75,
   'num_misses': 337,
   'num_switches': 1,
   'num_fragmentations': 36,
   'mota': 0.9182825484764543,
   'motp': 0.5552665799593458,
   'num_transfer': 0,
   'num_ascend': 1,
   'num_migrate': 0}}}